In [1]:
import easyocr
import os
import torch
import matplotlib.pyplot as plt
import random
import glob
from tqdm import tqdm
import numpy as np
import time
from mmocr_eval import eval_ocr_metric

In [2]:
#!pip install matplotlib

In [3]:
torch.cuda.is_available()

True

In [4]:
start = time.time()
reader = easyocr.Reader(lang_list = ['en','th'],
                        model_storage_directory = os.path.join('my_model','model'),
                        user_network_directory = os.path.join('my_model','user_network'),
                        recog_network  ='star_net',
                        gpu = True)
# reader = easyocr.Reader(lang_list = ['en','th'],gpu = True )
end = time.time()
print(end - start)
                        

star_net
my_model\user_network\star_net.yaml
Transformation: TPS
FeatureExtraction: ResNet
SequenceModeling: BiLSTM
Prediction: CTC
5.598895072937012


In [5]:
# plt.rcParams['font.family'] = 'TH Sarabun New'
# plt.rcParams['xtick.labelsize'] = 20.0
# plt.rcParams['ytick.labelsize'] = 20.0
# columns = 1
# rows = 1
# fig = plt.figure(figsize=(15, 15))
# ax = []
# for i in range(columns*rows):
#     # random_file_name = random.sample(os.listdir(os.path.join('trainer/all_data/testing/testing')),1)[0]
#     # random_file_path = os.path.join('trainer/all_data/testing/testing',random_file_name)
#     #random_file_name = random.sample(os.listdir(os.path.join('trainer/all_data/testing/testing')),1)[0]
#     random_file_name = os.listdir(os.path.join('trainer/all_data/testing/testing'))[2]
#     random_file_path = os.path.join('trainer/all_data/testing/testing',random_file_name)
#     img = plt.imread(random_file_path)
#     result = reader.readtext(random_file_path) 
#     ax.append(fig.add_subplot(rows, columns, i+1) )
#     try:
#         ax[-1].set_title(f'{random_file_name}predict: {result[0][1]}',fontsize=20)
#     except:
#         ax[-1].set_title('{random_file_name} No result',fontsize=20)
#     plt.imshow(img)
# fig.tight_layout() 
# plt.show()

In [6]:
filenames = glob.glob(os.path.join('trainer/all_data/testing/testing','*.jpg'))
label_path = 'trainer/all_data/testing/testing/label.txt'
with open(label_path, encoding='utf8') as file:
    label = np.loadtxt(file,dtype=str)
word_correct = 0
filenames = sorted(filenames, key= lambda x: int(x.split('/')[-1].split('\\')[1].split('.')[0]))
start = time.time()
# filenames = filenames[:10]
pred = []
gt = [] # ground truth
for i in tqdm(range(len(filenames))):
    label_img = label[i][1]

    result = reader.readtext(filenames[i], batch_size = 8)
    try:
        pred.append(result[0][1])
        gt.append(label_img)
    except:
        pred.append('')
        gt.append(label_img)
end = time.time()
output = eval_ocr_metric(pred, gt)
for k,v in output.items():
    print(f'{k}: {v}')
print(f'time: {end - start}')

  0%|          | 0/1000 [00:00<?, ?it/s]c:\Users\PIYAWAT\miniconda3\lib\site-packages\torch\nn\functional.py:4193: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
100%|██████████| 1000/1000 [02:05<00:00,  7.94it/s]

char_recall: 0.0682
char_precision: 0.2544
word_acc: 0.0
word_acc_ignore_case: 0.0
word_acc_ignore_case_symbol: 0.058
1-N.E.D: 0.0927
time: 125.96733689308167
